In [1]:
import data_preprocessing

In [2]:
# how to Daten einlesen
"""
Das Einlesen von Beschleunigungsdaten funktioniert gut. Der Code zum Einlesen von Gyroscopdaten muss noch dementsprechend angepasst werden.
"""
data = data_preprocessing.extract_accelerometer_data(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Kinnhacken.csv')

In [3]:
# hier ein Blick auf die Daten
print(data.head())

      timestamp         x         y         z
0  0.000000e+00  0.739594  0.358810 -0.562973
1  9.969000e+06  0.728119  0.300812 -0.555084
2  1.993700e+07  0.735657  0.260269 -0.555359
3  2.990600e+07  0.740021  0.261002 -0.569962
4  3.987500e+07  0.738632  0.268356 -0.582336


In [4]:
# autolabeling
"""
Für jeden Datensatz von den Sätzen mit hundert mal den kann jetzt einfach automatisch gelabeled werden. Man muss nur ggf. an den Parametern schrauben und am Anfang und Ende der Daten was abschneiden (data[550:] z.B.). Schaut gerne auch nochmal im Genaueren auf die Funktionsweise von autolabel. In kurz werden Hochpunkte im Datensatz identifiziert, um so die einzelnen Schläge als Dictionary in einer Liste zu speichern. 
"""
labeled_data = data_preprocessing.auto_labeling(data,  3, 140, 'Kinnhaken')

     index     timestamp          x         y         z
0      108  1.076611e+09  -6.170441  3.703735  3.569641
1      250  2.492154e+09  -1.506287  6.646332  3.246185
2      404  4.027319e+09  -0.983246  7.832825  4.251801
3      564  5.622294e+09  -2.752594  7.483246  2.832886
4      723  7.207298e+09  -1.463287  7.089508  4.322342
..     ...           ...        ...       ...       ...
96   18069  1.801200e+11  -6.060135  7.219940  4.491608
97   18266  1.820838e+11  -6.266785  8.120041  4.844223
98   18449  1.839081e+11  -6.030609  6.740952  4.870682
99   18638  1.857921e+11  -7.583420  7.285858  5.100525
100  18849  1.878955e+11 -11.425110  7.314713  6.263351

[101 rows x 5 columns]
179
0. Durchlauf: start:0, ende:179
327
1. Durchlauf: start:180, ende:327
484
2. Durchlauf: start:328, ende:484
644
3. Durchlauf: start:485, ende:644
806
4. Durchlauf: start:645, ende:806
972
5. Durchlauf: start:807, ende:972
1141
6. Durchlauf: start:973, ende:1141
1312
7. Durchlauf: start:1142, ende:13

In [5]:
# Prüfung ob labeling geklappt hat
"""
Hier wird geprüft, ob das Labeling geklappt hat. Dazu wird ein Plot erstellt. Hier kann man erkennen, ob das auto_labeling sauber die einzelnen Schläge identifiziert hat. Also immer zum Check machen!!
"""
#for i in range(0, len(labeled_data)):
    #data_preprocessing.plot_ds_element(labeled_data[i])

'\nHier wird geprüft, ob das Labeling geklappt hat. Dazu wird ein Plot erstellt. Hier kann man erkennen, ob das auto_labeling sauber die einzelnen Schläge identifiziert hat. Also immer zum Check machen!!\n'

In [6]:
# Normalisierung der Daten
"""
Gerne mit periodLengthMS und sampling sampleRateUS rumspielen. Den Effekt von den Variablen könnt ihr in smartPunch_tutorial_ManipulatePeriodLength.ipynb sehen.  
"""

periodLengthMS = 1500
sampleRateUS = 1000

ds = data_preprocessing.jsonData_to_dataset_in_timedifference_us(labeled_data)

df_new = data_preprocessing.normate_dataset_period(periodLengthMS, sampleRateUS, ds)

Starting new normalization/interpolation...
progress: 0.99 %
progress: 1.98 %
progress: 2.97 %
progress: 3.96 %
progress: 4.95 %
progress: 5.94 %
progress: 6.93 %
progress: 7.92 %
progress: 8.91 %
progress: 9.90 %
progress: 10.89 %
progress: 11.88 %
progress: 12.87 %
progress: 13.86 %
progress: 14.85 %
progress: 15.84 %
progress: 16.83 %
progress: 17.82 %
progress: 18.81 %
progress: 19.80 %
progress: 20.79 %
progress: 21.78 %
progress: 22.77 %
progress: 23.76 %
progress: 24.75 %
progress: 25.74 %
progress: 26.73 %
progress: 27.72 %
progress: 28.71 %
progress: 29.70 %
progress: 30.69 %
progress: 31.68 %
progress: 32.67 %
progress: 33.66 %
progress: 34.65 %
progress: 35.64 %
progress: 36.63 %
progress: 37.62 %
progress: 38.61 %
progress: 39.60 %
progress: 40.59 %
progress: 41.58 %
progress: 42.57 %
progress: 43.56 %
progress: 44.55 %
progress: 45.54 %
progress: 46.53 %
progress: 47.52 %
progress: 48.51 %
progress: 49.50 %
progress: 50.50 %
progress: 51.49 %
progress: 52.48 %
progress: 53

In [8]:
# Daten anschauen
print(df_new)

[             x         y         z  timestamp      label
0     0.762938  0.303203 -0.417425          0  Kinnhaken
1     0.763638  0.304910 -0.416414       1000  Kinnhaken
2     0.764289  0.306692 -0.415369       2000  Kinnhaken
3     0.764908  0.308554 -0.414301       3000  Kinnhaken
4     0.765504  0.310484 -0.413201       4000  Kinnhaken
...        ...       ...       ...        ...        ...
1496  0.756386  0.061219 -0.505723    1496000  Kinnhaken
1497  0.757806  0.057866 -0.507820    1497000  Kinnhaken
1498  0.758976  0.054962 -0.509515    1498000  Kinnhaken
1499  0.759906  0.052480 -0.510770    1499000  Kinnhaken
1500  0.760630  0.050343 -0.511791    1500000  Kinnhaken

[1501 rows x 5 columns],              x         y         z  timestamp      label
0     1.055252 -0.097946 -0.413071          0  Kinnhaken
1     1.053092 -0.100836 -0.397170       1000  Kinnhaken
2     1.050521 -0.103655 -0.382477       2000  Kinnhaken
3     1.047573 -0.106373 -0.368991       3000  Kinnhaken
4   